In [100]:
import far_ho as far
import tensorflow as tf
import far_ho.examples as far_ex

import os

In [133]:
tf.reset_default_graph()
ss = tf.InteractiveSession()

In [134]:
x = tf.placeholder(tf.float32, shape=(None, 28**2), name='x')
y = tf.placeholder(tf.float32, shape=(None, 10), name='y')

In [135]:
# load a small portion of mnist data
datasets = far_ex.mnist(folder=os.path.join(os.getcwd(), 'MNIST_DATA'), partitions=(.1, .1,))
datasets = far_ex.Datasets.from_list(datasets)

Extracting C:\Users\lfranceschi\PycharmProjects\FAR-HO\far_ho\examples\MNIST_DATA\train-images-idx3-ubyte.gz
Extracting C:\Users\lfranceschi\PycharmProjects\FAR-HO\far_ho\examples\MNIST_DATA\train-labels-idx1-ubyte.gz
Extracting C:\Users\lfranceschi\PycharmProjects\FAR-HO\far_ho\examples\MNIST_DATA\t10k-images-idx3-ubyte.gz
Extracting C:\Users\lfranceschi\PycharmProjects\FAR-HO\far_ho\examples\MNIST_DATA\t10k-labels-idx1-ubyte.gz
datasets.redivide_data:, computed partitions numbers - [0, 7000, 14000, 70000] len all 70000 DONE


In [136]:
# build up a feddforward NN calssifier
import tensorflow.contrib.layers as tcl
with tf.variable_scope('model'):
    h1 = tcl.fully_connected(x, 300)
    out = tcl.fully_connected(h1, datasets.train.dim_target)
    print('Ground model weights (parameters)')
    [print(e) for e in tf.model_variables()];
with tf.variable_scope('inital_weight_model'):
    h1_hyp = tcl.fully_connected(x, 300,
                                 variables_collections=[far.utils.GraphKeys.HYPERPARAMETERS, 
                                                       far.utils.GraphKeys.GLOBAL_VARIABLES], 
                                trainable=False)
    out_hyp = tcl.fully_connected(h1_hyp, datasets.train.dim_target,
                                 variables_collections=[far.utils.GraphKeys.HYPERPARAMETERS, 
                                                       far.utils.GraphKeys.GLOBAL_VARIABLES],
                                 trainable=False)
    print('Initial model weights (hyperparameters)')
    [print(e) for e in far.utils.hyperparameters()];
#     far.utils.remove_from_collection(far.GraphKeys.MODEL_VARIABLES, *far.utils.hyperparameters())

Ground model weights (parameters)
<tf.Variable 'model/fully_connected/weights:0' shape=(784, 300) dtype=float32_ref>
<tf.Variable 'model/fully_connected/biases:0' shape=(300,) dtype=float32_ref>
<tf.Variable 'model/fully_connected_1/weights:0' shape=(300, 10) dtype=float32_ref>
<tf.Variable 'model/fully_connected_1/biases:0' shape=(10,) dtype=float32_ref>
Initial model weights (hyperparameters)
<tf.Variable 'inital_weight_model/fully_connected/weights:0' shape=(784, 300) dtype=float32_ref>
<tf.Variable 'inital_weight_model/fully_connected/biases:0' shape=(300,) dtype=float32_ref>
<tf.Variable 'inital_weight_model/fully_connected_1/weights:0' shape=(300, 10) dtype=float32_ref>
<tf.Variable 'inital_weight_model/fully_connected_1/biases:0' shape=(10,) dtype=float32_ref>


In [137]:
# get an hyperparameter for weighting the examples for the inner objective loss (training error)
weights = far.get_hyperparameter('ex_weights', tf.zeros(datasets.train.num_examples))

In [138]:
# build loss and accuracy 
# inner objective (training error), weighted mean of cross entropy errors (with sigmoid to be sure is > 0)
with tf.name_scope('errors'):
    tr_loss = tf.reduce_mean(tf.sigmoid(weights)*tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out))
    # outer objective (validation error) (not weighted)
    val_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(out, 1)), tf.float32))

In [139]:
# optimizers
# get an hyperparameter for the learning rate
lr = far.get_hyperparameter('lr', 0.01)
io_optim = far.GradientDescentOptimizer(lr)  # for training error minimization an optimizer from far_ho is needed
oo_optim = tf.train.AdamOptimizer()  # for outer objective optimizer all optimizers from tf are valid

In [140]:
print('hyperparameters to optimize')
[print(h) for h in far.utils.hyperparameters()];

hyperparameters to optimize
<tf.Variable 'inital_weight_model/fully_connected/weights:0' shape=(784, 300) dtype=float32_ref>
<tf.Variable 'inital_weight_model/fully_connected/biases:0' shape=(300,) dtype=float32_ref>
<tf.Variable 'inital_weight_model/fully_connected_1/weights:0' shape=(300, 10) dtype=float32_ref>
<tf.Variable 'inital_weight_model/fully_connected_1/biases:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'ex_weights:0' shape=(7000,) dtype=float32_ref>
<tf.Variable 'lr:0' shape=() dtype=float32_ref>


In [141]:
# build hyperparameter optimizer
farho = far.HyperOptimizer()
run = farho.minimize(val_loss, oo_optim, tr_loss, io_optim, 
                     init_dynamics_dict={v: h for v, h in zip(tf.model_variables(), far.utils.hyperparameters()[:4])})

In [142]:
# run hyperparameter optimization 
T = 100 # performs 100 iteraitons of gradient descent on the training error 
# get data suppliers (could also be stochastic for SGD)
tr_supplier = datasets.train.create_supplier(x, y)
val_supplier = datasets.validation.create_supplier(x, y)
tf.global_variables_initializer().run()

print('training accuracy', accuracy.eval(tr_supplier()))
print('validation accuracy', accuracy.eval(val_supplier()))
print('-'*50)


for _ in range(10):
    run(T, inner_objective_feed_dicts=tr_supplier, outer_objective_feed_dicts=val_supplier)
    print('training accuracy', accuracy.eval(tr_supplier()))
    print('validation accuracy', accuracy.eval(val_supplier()))
    print('learning rate', lr.eval())
    print('norm of examples weight', tf.norm(weights).eval())
    print('-'*50)

    

training accuracy 0.12
validation accuracy 0.117143
--------------------------------------------------
training accuracy 0.482857
validation accuracy 0.454857
learning rate 0.011
norm of examples weight 0.0796879
--------------------------------------------------
training accuracy 0.611857
validation accuracy 0.589286
learning rate 0.0120009
norm of examples weight 0.152052
--------------------------------------------------
training accuracy 0.662714
validation accuracy 0.645
learning rate 0.0129937
norm of examples weight 0.2271
--------------------------------------------------
training accuracy 0.693571
validation accuracy 0.681714
learning rate 0.0139686
norm of examples weight 0.302777
--------------------------------------------------
training accuracy 0.72
validation accuracy 0.711857
learning rate 0.0149181
norm of examples weight 0.377813
--------------------------------------------------
training accuracy 0.766571
validation accuracy 0.759714
learning rate 0.0158564
norm of e